# Hacking Around Enumeration
by Mark Geyzer

<mark.geyzer@gmail.com> 


## Why Did I Need It?
Your REST request returned the response containing the following JSON

In [ ]:
resp = {'green': 'good',
        'red': 'bad'} 

And then you mistype the key

In [ ]:
resp['gren']

And imagine have you have hundreds of possible keys

On the fly - is the bug at the server side, or at your client?

## My Use Case

- I was working on a project with zillion string literals.

- Shame on me - I did not know about Python _enum_ then

__So - I just did it__

But now that I have learned the ugly truth - _enum_ exists.... 

## Why Did I Keep It?
(Besides the fact that it's **my precious**)

In [ ]:
import collections
import re
from operator import itemgetter

def enum_factory_currying(value_converter=None, 
                          field_converter=str.upper, 
                          ordered=False):
    """
    Wraps creation of enum-like constants container - with conversion function(s)
    
    :param value_converter: function to convert values - no conversion on default
    :param field_converter: function to convert field names - uppercase on default
    :param ordered: on True, order fields by values
    :return: function that builds container
    """
    def _tokenizer(word):
        """
        "Tokenize" string - convert to valid Python name
        
        :param word: word to tokenize
        :return: tokenised string value
        """
        return field_converter(re.sub('\W+', '_', word))

    def field_by_value(self, value):
        """
        Get mapper's symbolic name from value
        
        :param value: value to locate
        :return: values symbolic name
        :raises: ValueError, if value does not belong to enum
        """
        field = next((k for k, v in self._asdict().items() if v == value), None)
        if field is not None:
            return field
        raise ValueError(f'Undefined value "{value}" in enumeration')
    
    def value_processor(value):
        """
        Processes enumerated value according to its type and value_converter
        Allows recursive creation of JSON objects
        
        :param value: value to add to enum
        :return: processed value
        """
        if isinstance(value, dict):
            if value_converter is not None:
                return value_converter(**value)
            return enum_factory(**value)
        return value_converter(value) if value_converter is not None else value
   
    def enum_namer(l_):
        """
        Generates unique name for enum object
        
        :param l_: list of field names
        :return: name
        """
        return '_ENUM{:x}'.format(abs(hash(tuple(l_))))
    
    def _enum_builder(*fields, **kw_consts):
        """
        Wraps Enum creation constants, use-cases below may be mixed
         - For list of strings - tokenized string -> string
         - For keyword pairs - tokenized field name -> value

         IMPORTANT! each string used as a field name must be tokenizable,
                    i.e. used as source for valid Python ID,
                    spaces and alphanumeric literals, 
                    first character - alpha or underscore

        :param fields: constant strings to be wrapped
        :param kw_consts: identifier-values pairs to be wrapped
        :return: Enum object
        """
        if kw_consts:
            kw_pairs = sorted(kw_consts.items(), key=itemgetter(1)) if ordered \
                        else kw_consts.items()
            kw_keys, kw_values = [list(s) for s in zip(*kw_pairs)]
        else:
            kw_keys = []
            kw_values = []

        fields = list(fields)
        field_names = [_tokenizer(f) for f in fields + kw_keys]
        enum_values = [value_processor(v) for v in fields + kw_values]
        enum_factory = collections.namedtuple(enum_namer(field_names), field_names)
        enum_factory.__call__ = field_by_value  # Reverse values by calling object

        return enum_factory(*enum_values)

    return _enum_builder


# Non-currying mapper - for convenience
enum_factory = enum_factory_currying()


# Incremental integer mapper - for numeric sequence creation
def sequence_factory_currying(value_converter=lambda x: x, field_converter=str.upper):
    """
    Wraps convenient creation of integer enum-like sequence container
    
    :param value_converter: function to convert values
    :return: function that builds container
    """
    def _sequence_builder(*symbolic_names, start_value=0, step=1):
        """
        Maps symbolic names to sequence of integers
        
        :param symbolic_names: names for mapping
        :param start_value: initial value of sequence
        :param step: sequence step
        :return: enum-like symbol-2-integer mapper
        """
        sequence_kwargs = dict((name, value * step) 
                               for value, name in 
                               enumerate(symbolic_names, start_value))
        return enum_factory_currying(value_converter=value_converter, 
                                     field_converter=field_converter, 
                                     ordered=True)(**sequence_kwargs)
    return _sequence_builder


# Non-currying mapper - for convenience
sequence_factory = sequence_factory_currying()


import configparser
def load_ini(source):
    """
    Loads INI file as 2-tiered JSON object
    
    :param source: configurations (as a string)
    :return: config object
    """
    def convert_config_value(value):
        """
        Conver configuration value to an appropriate type
        
        :param value: value as received from configuration file
        :return: converted value
        """
        boolean_value = {'true': True, 'false': False}.get(value.lower())
        if boolean_value is not None:
            return boolean_value
        
        for numeric_type in (int, float):
            try:
                converted_value = numeric_type(value)
                return converted_value
            except ValueError:
                pass
        return value
    
    config_obj = configparser.ConfigParser()
    config_obj.read_string(source)
    ini_value_converter = enum_factory_currying(value_converter=convert_config_value,
                                                field_converter=str.lower)
    config_loader = enum_factory_currying(value_converter=ini_value_converter)

    return config_loader(**{name: dict(section) 
                            for name, section in config_obj.items()})


import json
def load_json(source, field_converter=str.upper):
    """
    Create immutable JSON object from source string
      
    :param source: serialized JSON
    :param field_converter: field names' converter
    :return: JSON object
    """
    config_object = json.loads(source)
    return enum_factory_currying(field_converter=field_converter)(**config_object)

## Quick Intro

##### Exhibit 1
Let's enumerate the `quick brown fox`

In [ ]:
import enum
class Fox(enum.Enum):
    QUICK = 'quick'
    BROWN = 'brown'
    FOX = 'fox'
    ACTION = 'jumps'
    TIMES = 8

Now I have painstakingly defined an _enum_ `Fox` with 5 attributes. But what if you could do it in a shorter version?

In [ ]:
FOX = enum_factory('quick', 'brown', 'fox', action='jumps', times=8)

**What's the difference?**

- more succinct

- used a function call instead of a class creation.

- positional arguments - when adding strings

- keyword arguments - when cannot use literal value as symbolic name

## What else is different?

In [ ]:
FOX.TIMES == Fox.TIMES

**Really?**

In [ ]:
print(*(type(v) for v in (FOX.TIMES, Fox.TIMES)))

Can we compare them? Yep, with some modification

In [ ]:
FOX.TIMES == Fox.TIMES.value

 (who wants to type extra?!)

Imagine that you want to know if value 8 is in `Fox`?

In [ ]:
8 in Fox

And what is the symbolic name of value 8? 

In [ ]:
dir(Fox)

You got the message. What about the door \#2?

In [ ]:
8 in FOX

In [ ]:
FOX(8)

Go, _enum_ ?!



##### Exhibit 2
Let us define a pure _int_ _enum_ - both by vanilla Python and by another function from the door \#2 (built on the base of the first)

In [ ]:
class Count(enum.IntEnum):
    ZERO = 0
    ONE = 1
    TWO = 2
    THREE = 3
    
COUNT = sequence_factory('zero', 'one', 'two', 'three')
print(COUNT.ONE == 1, Count.ONE == 1) 

At least, comparison is better, but is not writing _enum_ a little more tedious?

But before going further down the rabbit hole, some explanation.

### What Hides Behind The Door #2?

- Take _namedtuples_

- Wrap with a set of higher order (currying) functions, AKA Y-combinator

*enum_factory*  and *sequence_factory* are shortcuts, hiding some parameters of the underlying higher-order functions. 

You can also expand shortcuts set for more specific cases.

## What Can You Do With It?
You have already met that pesky `quick brown fox`

In [ ]:
print(FOX.QUICK, FOX.BROWN, FOX.FOX, FOX.ACTION, FOX.TIMES, 'times')

###### and name from value

In [ ]:
FOX(8)

###### clear source of error

In [ ]:
FOX.BRON

### How About JSON Objects?


In [ ]:
girl = {
    'Is there': {'anybody': 'going', 'to listen': 'to my story'},
    'all': {'about': 'the girl', 'who came': 'to stay'}
}

In [ ]:
GIRL = enum_factory(**girl)

In [ ]:
GIRL.ALL.ABOUT

In [ ]:
GIRL.IS_THERE.ANYBODY

### Throw In Some Functions
After all, they are first class citizens

In [ ]:
S = enum_factory(u=str.upper, l=str.lower, t=str.title)

In [ ]:
S.T('what can i do')

### Map and Enumerate
You can enumerate non-scalar objects - and throw _map_ -like action in the mix.

E.g., set of _pathlib.Path_ objects. And here the "hidden" currying form comes handy, offering value conversion on the way

In [ ]:
import pathlib
PATHS = enum_factory_currying(value_converter=pathlib.Path)(scripts='/usr/bin/src', 
                                                            home='/home/user',
                                                            data='/home/data')

In [ ]:
PATHS.SCRIPTS

### Let's play around with some spy stuff


In [ ]:
import string, itertools
chars = string.ascii_lowercase + string.ascii_uppercase
def encode_word(word, offsets=(-1, 1)):
    return ''.join(chars[(chars.index(char) + offs) % 52] 
                   for char, offs in zip(word, itertools.cycle(offsets)))

SECRET = enum_factory_currying(value_converter=encode_word, field_converter=str.title)\
                              ('Quick', 'brown', 'fox', action='jumps')

In [ ]:
SECRET.Fox

In [ ]:
SECRET.Quick

### Play Around With Sequences
Simple sequence

In [ ]:
import string
ALPHA = sequence_factory(*list(string.ascii_lowercase), start_value=1)

In [ ]:
ALPHA.H

Sequence with a twist

In [ ]:
# This is the currying form
SZ = sequence_factory_currying(value_converter=lambda v: 1024 ** v)\
                              ('B', 'KB', 'MB', 'GB', 'TB')

In [ ]:
SZ.MB

In [ ]:
SZ(1024)

Remember I promised you goodies?

### Parse INI configurations

Ever done that? It may be annoying - if you do it on your own!


Well, you don't have to any more - that is wrapped for you too!

In [ ]:
config_file = '''
[owner]
name = John Doe
organization = Acme Widgets Inc.

[database]
server = 192.0.2.62     
port= 143
file = payroll.dat
'''

In [ ]:
CFG = load_ini(config_file)

In [ ]:
CFG.OWNER.name

In [ ]:
CFG.DATABASE.file

## The Implementation
It is more than one screen long, so you can take a look later.

Here are the APIs

### Base

```python
# Currying function
enum_factory_currying(
    value_converter=None, field_converter=str.upper, ordered=False)\
    (*fields, **kw_consts)
# Shortcut
enum_factory(*fields, **kw_consts)  

```

### Sequences

```python
# Currying function
sequence_factory_currying(
    value_converter=lambda x: x, field_converter=str.upper)\
    (*symbolic_names, start_value=0, step=1)
# Shortcut
sequence_factory(*symbolic_names, start_value=0, step=1) 
```

### Expansions

```python
load_ini(source)
load_json(source, field_converter=str.upper)
```

## Summary

##### Door #1
```python
class Colours(enum.Enum):
    RED = 'red'
    GREEN = 'green'
    MAGENTA = 'purpl-ish?'

class COLOURS(enum.IntEnum):
    RED = 1
    GREEN = 2
```

##### Door #2
```python
Colours = enum_factory('red', 'green', magenta='purpl-ish?')
COLOURS = sequence_factory('red', 'green', start=1)
```

### What Do We Get Behind Door \#2?

- Brevity

- Functional API

- A freedom to mix _\*args_ and _\*\*kwargs_ as we see fit

- Easily mix values of any type - including functions

- _enum_ attributes retain their type
 - try to use native *enum* as *pandas.DataFrame* indicies!

- Customizable expansions

## Q & A?
The code can be found at https://github.com/geyzer63/Enumerator